In [1]:
import perceval as pcvl
from perceval.utils import StateVector
import perceval.components as comp
import perceval.simulators as sim
import perceval as pcvl
from perceval.utils.postselect import PostSelect
import numpy as np
results = []

In [2]:

W_8 = StateVector([1,0,0,0,0,0,0,0]) + StateVector([0,1,0,0,0,0,0,0]) + StateVector([0,0,1,0,0,0,0,0]) +StateVector([0,0,0,1,0,0,0,0])+StateVector([0,0,0,0,1,0,0,0])+StateVector([0,0,0,0,0,1,0,0])+StateVector([0,0,0,0,0,0,1,0])+StateVector([0,0,0,0,0,0,0,1])
W_8_tensor = W_8 * W_8
loss = 0.01
deviation = 0.004
loss_array = np.random.normal(loss, deviation, 16)
lossy_channel = (pcvl.Processor("SLOS", 16)
             .add(0, pcvl.LC(loss_array[0]))
             .add(1, pcvl.LC(loss_array[1]))
             .add(2, pcvl.LC(loss_array[2]))
             .add(3, pcvl.LC(loss_array[3]))  
             .add(4, pcvl.LC(loss_array[4]))
             .add(5, pcvl.LC(loss_array[5]))
             .add(6, pcvl.LC(loss_array[6]))
             .add(7, pcvl.LC(loss_array[7]))  
             .add(8, pcvl.LC(loss_array[8]))  
             .add(9, pcvl.LC(loss_array[9]))  
             .add(10, pcvl.LC(loss_array[10]))  
             .add(11, pcvl.LC(loss_array[11]))  
             .add(12, pcvl.LC(loss_array[12]))
             .add(13, pcvl.LC(loss_array[13]))  
             .add(14, pcvl.LC(loss_array[14]))  
             .add(15, pcvl.LC(loss_array[15]))
             )


In [3]:
lossy_channel.min_detected_photons_filter(0)
lossy_channel.with_input(W_8_tensor)
imperfect_sampler = pcvl.algorithm.Sampler(lossy_channel)
output = imperfect_sampler.probs()["results"]
print(len(output))
pcvl.pdisplay(output)

81


In [4]:
print(len(output), "Ideal case: ",56*0.015625)
total_success = 0
def remove_collisions(output):
    filtered_output = {}
    # Assume each party has 2 modes, total of 8 parties → len(key) = 16
    for key, value in output.items():
        valid = True
        for i in range(8):
            if key[i] == 1 and key[i + 8] == 1:
                valid = False
                break
        if valid:
            filtered_output[key] = value
    return filtered_output


filtered_output = remove_collisions(output)
def count_2_photon_cases(filtered_output):
    count = 0
    two_photon_dict = {}
    total_value = 0
    for key, value in filtered_output.items():
        if sum(key) == 2:
            total_value += value
            two_photon_dict[key] = value
    return total_value, two_photon_dict

print(len(filtered_output))
success_rate, success_dict = count_2_photon_cases(filtered_output)

variance = 0
for key, value in success_dict.items():
    qi = value / success_rate
    variance_term = ((qi -  1/56)**2)
    variance += variance_term
print("Distance from uniformity: ", variance)
print("success rate: ", success_rate, "number of useful patterns: ", len(success_dict), "success rate check:", sum(success_dict.values()))
print("dict of successful patterns: ", success_dict)
results.append([loss, deviation, success_rate, variance])

81 Ideal case:  0.875
73
Distance from uniformity:  7.924321698713616e-07
success rate:  0.8599901157116575 number of useful patterns:  56 success rate check: 0.8599901157116575
dict of successful patterns:  {|1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0>: 0.015218018459123722, |1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0>: 0.015263685600611381, |0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0>: 0.015499055782532162, |0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0>: 0.015573726264228501, |0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0>: 0.015346041034474851, |0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0>: 0.015292653261040727, |0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1>: 0.015382957217343009, |0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0>: 0.015431444162700094, |0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0>: 0.015493565224393947, |0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0>: 0.015287235821230491, |0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0>: 0.015379625778184646, |0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0>: 0.015178737327838667, |0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0>: 0.01528542753520114, |0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0>: 0.01528121043172